In [ ]:
# import os, re
# if "COLAB_" not in "".join(os.environ.keys()):
#     !pip install unsloth
# else:
#     # Do this only in Colab notebooks! Otherwise use pip install unsloth
#     import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
#     xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
#     !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
#     !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
#     !pip install --no-deps unsloth
# !pip install transformers==4.56.2
# !pip install --no-deps trl==0.22.2
# !pip install snac

# Baseline

In [ ]:
# use from teminal to dowload the model without extra packages
# !huggingface-cli download sesame/csm-1b --exclude "ckpt.pt" "model.safetensors"

In [ ]:
import uuid
import torch
import librosa
import subprocess
from IPython.display import Audio
from transformers import CsmForConditionalGeneration, AutoProcessor

## sesame/csm-1b

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = AutoProcessor.from_pretrained("sesame/csm-1b")
model = CsmForConditionalGeneration.from_pretrained("sesame/csm-1b", device_map=device)

def load_audio_24k(path):
    # mono, float32 numpy array at 24 kHz (required)
    y, sr = librosa.load(path, sr=24000, mono=True)
    return y

model

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.69s/it]


CsmForConditionalGeneration(
  (lm_head): Linear(in_features=2048, out_features=2051, bias=False)
  (embed_text_tokens): Embedding(128256, 2048)
  (backbone_model): CsmBackboneModel(
    (embed_tokens): CsmBackboneModelEmbeddings(
      (embed_audio_tokens): Embedding(65632, 2048)
    )
    (layers): ModuleList(
      (0-15): 16 x CsmDecoderLayer(
        (self_attn): CsmAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): CsmMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        

In [4]:
conversation = [
    {
        "role": "0",
        # "content": [{"type": "text", "text": 'Il ragazzo corre rapido per raggiungere la roccia.'}]
        "content": [{"type": "text", "text": 'The boy run fast to get to the rock.'}]
    }
]

inputs = processor.apply_chat_template(
    conversation,
    tokenize=False, #True,
    return_dict=True,
)

print(inputs)

inputs = processor.apply_chat_template(
    conversation,
    tokenize=True,
    return_dict=True,
).to(device)

audio = model.generate(**inputs, output_audio=True)
Audio(audio[0].cpu().numpy(), rate=24000)

<|begin_of_text|>[0]The boy run fast to get to the rock.<|end_of_text|>


In [6]:
conversation = [
    {
        "role": "0",
        "content": [
            {"type": "text", "text": "Ora che hai fatto il pieno di informazioni utili, prenditi un momento per pensarci su e rispondi a questa domanda."},
            {"type": "audio", "path": "data/audio/M01UD01p10.wav"}
        ]
    },
    {
        "role": "0",
        "content": [
            {"type": "text", "text": "berlusconi? Quiz, tv, paghe da fame"}
        ]
    }
]

inputs = processor.apply_chat_template(
    conversation,
    tokenize=False, #True,
    return_dict=True,
)

print(inputs)


inputs = processor.apply_chat_template(
    conversation,
    tokenize=True,
    return_dict=True,
).to(model.device)

audio = model.generate(**inputs, output_audio=True)
# processor.save_audio(audio, "example_with_context.wav")
Audio(audio[0].cpu().numpy(), rate=24000)



<|begin_of_text|>[0]Ora che hai fatto il pieno di informazioni utili, prenditi un momento per pensarci su e rispondi a questa domanda.<|end_of_text|><|AUDIO|><|audio_eos|><|begin_of_text|>[0]berlusconi? Quiz, tv, paghe da fame<|end_of_text|>


## canopylabs/3b-es_it-ft-research_release

In [ ]:
# huggingface-cli download canopylabs/3b-es_it-ft-research_release \
#   --exclude "optimizer.pt" "scheduler.pt" "trainer_state.json" "training_args.bin" "rng_state.pth"

# huggingface-cli download canopylabs/3b-es_it-pretrain-research_release \
#   --exclude "scheduler.pt" "trainer_state.json" "training_args.bin" "rng_state.pth" "pytorch_model_fsdp.bin" "optimizer.bin"


In [2]:
import torch
import librosa

from snac import SNAC
from IPython.display import display
from transformers import AutoModelForCausalLM, AutoTokenizer

snac_model = SNAC.from_pretrained("hubertsiuzdak/snac_24khz")
snac_model = snac_model.to("cpu")

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "canopylabs/3b-es_it-ft-research_release"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.46s/it]


In [ ]:
# Choose the target voice (prefix used inside the prompt)
chosen_voice = "giulia" # pietro, giulia, carlo

pangrammi = [
  'berlusconi? Quiz, tv, paghe da fame'
]

rolle_r = [
'Il ragazzo corre rapido per raggiungere la roccia.',
'A Roma, il rumore del tram rimbomba tra le strade.',
'La terra rossa è ricca di minerali rari.'
]

double_consonants = [
'La palla cadde vicino alla cassa della villa.',
'Ho fatto una focaccia di pasta con una spatola piccola.',
'La gatta attende la pappa sotto il tetto.',
'Ho aspettatto otto minuti prima di attraversare.',
'Fratelli, non lottate per l’ombrello rotto!',
'La battutta del batterista era perfetta e potente.',
'La cattedra è aspettata da tutti gli studentti attenti.'
]

gn = [
'La portata di gnocchi era stranamente buona.',
'Nel bagno, il ragno si nasconde dietro lo specchio.',
'La lasagna della nonna è un sogno ogni domenica.',
]

gli = [
'I figli di Guigliaelmo raccolgono aglio e olio.',
'La famiglia si sveglia presto a luglio.',
'Le foglie brillano sotto gli sbagli di luce.'
]

open_vs_closed_vowels = [
'Mangia una pesca, ma pesca anche nel fiume.',
'La botte è piena, ma non dare botte alla porta.'
]

z = [
'La pizza in piazza è pazzesca!',
'Grazie, la zuppa è speziata ma deliziosa.',
'Zero zanzare in zoo, che sorpresa straordinaria!'
]

il_lonfo = [
"""Il Lonfo non vaterca né gluisce
e molto raramente barigatta,
ma quando soffia il bego a bisce bisce
sdilenca un poco e gnagio s'archipatta.
È frusco il Lonfo!
""".replace('\n', ' ')
]


prompts = pangrammi

prompts = [f"{chosen_voice}: " + p for p in prompts]

all_input_ids = []

# Tokenize text prompts into input IDs
for prompt in prompts:
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids
  all_input_ids.append(input_ids)

# Special tokens used by the model
start_token = torch.tensor([[128259]], dtype=torch.int64) # Start of human
end_tokens = torch.tensor([[128009, 128260]], dtype=torch.int64) # End of text, End of human

# Add special tokens to each input sequence
all_modified_input_ids = []
for input_ids in all_input_ids:
  # Final structure: SOH + TEXT + (EOT, EOH)
  modified_input_ids = torch.cat([start_token, input_ids, end_tokens], dim=1)
  all_modified_input_ids.append(modified_input_ids)

# PAD all sequences to the same max length
all_padded_tensors = []
all_attention_masks = []
max_length = max([modified_input_ids.shape[1] for modified_input_ids in all_modified_input_ids])
for modified_input_ids in all_modified_input_ids:
  padding = max_length - modified_input_ids.shape[1]

  # Pad on the left using PAD token 128263
  padded_tensor = torch.cat([torch.full((1, padding), 128263, dtype=torch.int64), modified_input_ids], dim=1)
  attention_mask = torch.cat([torch.zeros((1, padding), dtype=torch.int64), torch.ones((1, modified_input_ids.shape[1]), dtype=torch.int64)], dim=1)
  all_padded_tensors.append(padded_tensor)
  all_attention_masks.append(attention_mask)

all_padded_tensors = torch.cat(all_padded_tensors, dim=0)
all_attention_masks = torch.cat(all_attention_masks, dim=0)

input_ids = all_padded_tensors.to(device)
attention_mask = all_attention_masks.to(device)

In [22]:
with torch.no_grad():
  generated_ids = model.generate(
      input_ids=input_ids,
      attention_mask=attention_mask,
      max_new_tokens=1200,
      do_sample=True,
      temperature=0.6,
      top_p=0.95,
      repetition_penalty=1.1,
      num_return_sequences=1,
      eos_token_id=128258,
)
  
generated_ids.shape

Setting `pad_token_id` to `eos_token_id`:128258 for open-end generation.


torch.Size([1, 295])

In [ ]:
# Special tokens used in the generated output
token_to_find = 128257              # <custom_token_1>: keep only tokens *after* the last occurrence
eos_token_id_to_remove = 128258     # Token to completely remove (padding/end marker)

# Find all positions where token_to_find appears
token_indices = (generated_ids == token_to_find).nonzero(as_tuple=True)
print(token_indices)

# If at least one marker token is found, crop everything before the last one
if len(token_indices[1]) > 0:
    last_occurrence_idx = token_indices[1][-1].item()
    cropped_tensor = generated_ids[:, last_occurrence_idx+1:]
else:
    cropped_tensor = generated_ids


processed_rows = []

# Remove all 'eos_token_id' tokens row-by-row
for row in cropped_tensor:
    masked_row = row[row != eos_token_id_to_remove]
    processed_rows.append(masked_row)

code_lists = []

# Convert processed token sequences into codec code groups
for row in processed_rows:
    row_length = row.size(0)

    # The codec expects groups of 7 tokens → trim to nearest multiple of 7
    new_length = (row_length // 7) * 7
    trimmed_row = row[:new_length]

    # Remove the global vocabulary offset used by the language model
    trimmed_row = [t - 128266 for t in trimmed_row]
    code_lists.append(trimmed_row)


def redistribute_codes(code_list):
  """
  Reconstructs SNAC codec layers from a flat sequence of 7 tokens per timestep.
  Each of the 7 tokens belongs to a different RVQ codebook with a different offset.
  """
  layer_1 = []
  layer_2 = []
  layer_3 = []
  for i in range((len(code_list)+1)//7):
    layer_1.append(code_list[7*i])
    layer_2.append(code_list[7*i+1]-4096)
    layer_3.append(code_list[7*i+2]-(2*4096))
    layer_3.append(code_list[7*i+3]-(3*4096))
    layer_2.append(code_list[7*i+4]-(4*4096))
    layer_3.append(code_list[7*i+5]-(5*4096))
    layer_3.append(code_list[7*i+6]-(6*4096))
  codes = [torch.tensor(layer_1).unsqueeze(0),
         torch.tensor(layer_2).unsqueeze(0),
         torch.tensor(layer_3).unsqueeze(0)]
  audio_hat = snac_model.decode(codes)
  return audio_hat

my_samples = []
for code_list in code_lists:
  samples = redistribute_codes(code_list)
  my_samples.append(samples)

(tensor([0], device='cuda:0'), tensor([20], device='cuda:0'))


In [13]:
from IPython.display import display, Audio
if len(prompts) != len(my_samples):
  raise Exception("Number of prompts and samples do not match")
else:
  for i in range(len(my_samples)):
    print(prompts[i])
    samples = my_samples[i]
    display(Audio(samples.detach().squeeze().to("cpu").numpy(), rate=24000))

giulia: berlusconi? Quiz, tv, paghe da fame


Zero-shot voice cloning seems broken atm... (see https://github.com/canopyai/Orpheus-TTS/issues/6)

# LoRA Finetune (Unsloth)

## Dataset preparation

In [1]:
import librosa, pathlib

total = 0
for wav in pathlib.Path("data/audio").glob("*.wav"):
    y, sr = librosa.load(wav, sr=None)
    total += len(y) / sr / 60

print(f"Total: ~{int(total)} min")

Total: ~10 min


### Whisper trascribe

In [ ]:
import os
import json
import whisper
from pathlib import Path

device = 'cuda'
model = whisper.load_model("turbo", device=device)

audio_paths = [Path(os.path.join('data/audio', p)) for p in os.listdir('data/audio')]
TRANSCRIPTS_DIR = Path("data/transcripts")
TRANSCRIPTS_DIR.mkdir(exist_ok=True)
CLIPS_ROOT = Path("data/clips")
CLIPS_ROOT.mkdir(exist_ok=True)


for audio_path in audio_paths:
    fname = audio_path.stem
    out_json = TRANSCRIPTS_DIR / f"{fname}_whisper.json"

    if out_json.exists():
        print(f"⏭️ Skipping transcription for {fname} (already exists)")
        continue

    print(f"\n🎤 Transcribing {fname} ...")

    result = model.transcribe(
        str(audio_path),
        word_timestamps=True,
        fp16=(device == "cuda"),
        verbose=False,
        language='it'
    )

    with out_json.open("w") as f:
        json.dump(result, f, indent=2)

    print("✅ Saved →", out_json)

print(f"\n🏁 Done! All transcripts in", TRANSCRIPTS_DIR)


⏭️ Skipping transcription for M01UD01p2 (already exists)
⏭️ Skipping transcription for M01UD01p5 (already exists)
⏭️ Skipping transcription for M01UD01p1 (already exists)
⏭️ Skipping transcription for M01UD01p8 (already exists)
⏭️ Skipping transcription for M01UD01p9 (already exists)
⏭️ Skipping transcription for M01UD01p4 (already exists)
⏭️ Skipping transcription for M01UD01p6 (already exists)
⏭️ Skipping transcription for M01UD01p3 (already exists)
⏭️ Skipping transcription for M01UD01p10 (already exists)
⏭️ Skipping transcription for M01UD01p7 (already exists)

🏁 Done! All transcripts in data/transcripts


### Dataset chunking

In [ ]:
sampling_rate = 24000
MAX_SEC = 30

def chunk_one(audio_path: Path, json_path: Path):
    """
    Process one audio file + its Whisper transcript.
    Returns a list of dicts: {audio, text, source}.
    Each dict corresponds to a clipped audio segment aligned to sentences.
    """

    # Load Whisper JSON transcript
    with open(json_path) as f:
        data = json.load(f)
    segments = data["segments"]

    # STEP 1 — Build word-level metadata
    # Whisper gives timestamps at the word level.
    # We reconstruct: 
    #    - the full text string
    #    - for each word: its char span and its time span
    #
    word_meta, full_text, char_pos = [], [], 0

    for seg in segments:
        for w in seg["words"]:
            tok = w["word"].strip()
            if not tok:
                continue

            start_char = char_pos
            end_char   = char_pos + len(tok)

            # Save mapping: characters → time
            word_meta.append({
                "char0": start_char,
                "char1": end_char,
                "t0": w["start"],
                "t1": w["end"]
            })

            # Build full text with spaces
            full_text.extend([tok, " "])
            char_pos = end_char + 1   # +1 because we added a space

    # Create the final text, trimming trailing space
    full_text = "".join(full_text).rstrip()

    # STEP 2 — Split the text into sentences using NLTK
    # We get sentence character spans (c0, c1).
    # Later, we use these spans to map sentences back to word-level timestamps.

    import nltk
    from nltk.tokenize import PunktSentenceTokenizer
    nltk.download("punkt", quiet=True)
    tokenizer = PunktSentenceTokenizer()

    # List of (char_start, char_end) for each detected sentence
    sent_spans = list(tokenizer.span_tokenize(full_text))

    sentences, w_idx = [], 0

    # For each sentence: determine which words fall into it → get start/end times
    for c0, c1 in sent_spans:

        # Find first word whose end char > c0
        first_w = w_idx
        while first_w < len(word_meta) and word_meta[first_w]["char1"] <= c0:
            first_w += 1

        # Find last word whose start char < c1
        last_w = first_w
        while last_w < len(word_meta) and word_meta[last_w]["char0"] < c1:
            last_w += 1

        if first_w == last_w:
            # Sentence spans no words (rare, but good safeguard)
            continue

        # Sentence time boundaries = first/last word timestamps
        s_time = word_meta[first_w]["t0"]
        e_time = word_meta[last_w - 1]["t1"]
        text   = full_text[c0:c1]

        sentences.append({"start": s_time, "end": e_time, "text": text})
        w_idx = last_w  # advance word index for efficiency

    # STEP 3 — Bundle sentences into max-length clips and export audio
    # Whisper sentences can be too short or too many.
    # We merge consecutive sentences so long as the resulting clip
    # does not exceed MAX_SEC seconds.
    #
    clip_rows, bundle = [], None

    def flush(b):
        """Write a bundled clip to disk and store metadata."""
        if b is None: 
            return

        st, et, tx, out_dir = b
        out_dir.mkdir(exist_ok=True, parents=True)

        clip_path = out_dir / f"clip_{uuid.uuid4().hex}.wav"

        # Use ffmpeg to slice the original audio
        subprocess.run([
            "ffmpeg", "-loglevel", "error", "-y",
            "-i", str(audio_path),
            "-ss", f"{st}", "-to", f"{et}",
            "-ar", str(sampling_rate), "-ac", "1",
            str(clip_path)
        ], check=True)

        clip_rows.append({
            "audio": str(clip_path),
            "text": tx.strip(),
            "source": "luca"
        })

    # Iterate all sentences and group them
    out_dir = CLIPS_ROOT / audio_path.stem
    for s in sentences:
        st, et, tx = s["start"], s["end"], s["text"]
        dur = et - st

        # Drop single-sentence clips that exceed the max duration
        if dur > MAX_SEC:
            print(f'Warning: skipped long sentence {dur}s')
            continue

        if bundle is None:
            # Start a new bundle
            bundle = [st, et, tx, out_dir]
            continue

        b_st, b_et, b_tx, _ = bundle

        # If adding the new sentence stays within MAX_SEC -> merge
        if (et - b_st) <= MAX_SEC:
            bundle = [b_st, et, b_tx + " " + tx, out_dir]
        else:
            # Otherwise output the current bundle and start a new one
            flush(bundle)
            bundle = [st, et, tx, out_dir]

    # Flush any remaining bundle
    flush(bundle)
    return clip_rows


# --------------------------------------------------------------------------
# PROCESS ALL FILES
# --------------------------------------------------------------------------
audio_paths = [Path(os.path.join('data/audio', p)) for p in os.listdir('data/audio')]
rows = []
for audio_path in audio_paths:
    stem = audio_path.stem
    json_path = TRANSCRIPTS_DIR / f"{stem}_whisper.json"

    if not json_path.exists():
        print(f"⚠️ {stem} → skipping (no transcript)")
        continue

    print(f"→ Chunking {stem} ...")
    clip_rows = chunk_one(audio_path, json_path)
    rows.extend(clip_rows)
    print(f"   {len(clip_rows)} clips")

print(f"\n✅ Created {len(rows)} total clips across ")

# --------------------------------------------------------------------------
# SAVE TO HF DATASET FORMAT
# --------------------------------------------------------------------------
from datasets import Dataset
ds = Dataset.from_list(rows)
ds.save_to_disk("data/dataset")

print(ds)


→ Chunking M01UD01p2 ...
   4 clips
→ Chunking M01UD01p5 ...
   4 clips
→ Chunking M01UD01p1 ...
   2 clips
→ Chunking M01UD01p8 ...
   3 clips
→ Chunking M01UD01p9 ...
   2 clips
→ Chunking M01UD01p4 ...
   2 clips
→ Chunking M01UD01p6 ...
   4 clips
→ Chunking M01UD01p3 ...
   5 clips
→ Chunking M01UD01p10 ...
   1 clips
→ Chunking M01UD01p7 ...
   2 clips

✅ Created 29 total clips across 


/home/lpozzi/Git/data-science-lectures/tts_ita/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Saving the dataset (1/1 shards): 100%|██████████| 29/29 [00:00<00:00, 3909.96 examples/s]

Dataset({
    features: ['audio', 'text', 'source'],
    num_rows: 29
})


### Tokenisation Function

In [4]:
import torch
import librosa
import torchaudio.transforms as T

from snac import SNAC
device = 'cuda'

snac_model = SNAC.from_pretrained("hubertsiuzdak/snac_24khz").to(device)
ds_sample_rate = 24000

# Tokenisation Function
def tokenise_audio(waveform):
    waveform = torch.from_numpy(waveform).unsqueeze(0)
    waveform = waveform.to(dtype=torch.float32)
    resample_transform = T.Resample(orig_freq=ds_sample_rate, new_freq=24000)
    waveform = resample_transform(waveform)

    waveform = waveform.unsqueeze(0).to(device)

    #generate the codes from snac
    with torch.inference_mode():
        codes = snac_model.encode(waveform)

    all_codes = []
    for i in range(codes[0].shape[1]):
        all_codes.append(codes[0][0][i].item()+128266)
        all_codes.append(codes[1][0][2*i].item()+128266+4096)
        all_codes.append(codes[2][0][4*i].item()+128266+(2*4096))
        all_codes.append(codes[2][0][(4*i)+1].item()+128266+(3*4096))
        all_codes.append(codes[1][0][(2*i)+1].item()+128266+(4*4096))
        all_codes.append(codes[2][0][(4*i)+2].item()+128266+(5*4096))
        all_codes.append(codes[2][0][(4*i)+3].item()+128266+(6*4096))

    return all_codes

### Map Tokenize

In [5]:
def add_codes(example):
    # Always initialize codes_list to None
    codes_list = None
    audio_path = example['audio']
    audio_array, sr = librosa.load(audio_path, sr=None)
    codes_list = tokenise_audio(audio_array)
    
    example["codes_list"] = codes_list

    return example

ds = ds.map(add_codes, remove_columns=["audio"])
ds

Parameter 'function'=<function add_codes at 0x7df2dc4d1480> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only shown once. Subsequent hashing failures won't be shown.
Map: 100%|██████████| 29/29 [00:06<00:00,  4.70 examples/s]


Dataset({
    features: ['text', 'source', 'codes_list'],
    num_rows: 29
})

### Load tokenizer

In [6]:
tokeniser_length = 128256
start_of_text = 128000
end_of_text = 128009

start_of_speech = tokeniser_length + 1
end_of_speech = tokeniser_length + 2

start_of_human = tokeniser_length + 3
end_of_human = tokeniser_length + 4

start_of_ai = tokeniser_length + 5
end_of_ai =  tokeniser_length + 6
pad_token = tokeniser_length + 7

audio_tokens_start = tokeniser_length + 10

tokenizer_name = "canopylabs/3b-es_it-ft-research_release"

from transformers import AutoTokenizer
import os
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
num_proc = os.cpu_count() - 2

ds = ds.filter(lambda x: x["codes_list"] is not None)
ds = ds.filter(lambda x: len(x["codes_list"]) > 0)
ds

Filter: 100%|██████████| 29/29 [00:00<00:00, 851.87 examples/s]


Dataset({
    features: ['text', 'source', 'codes_list'],
    num_rows: 29
})

### Create Input Ids

In [7]:
def remove_duplicate_frames(example):
    vals = example["codes_list"]
    if len(vals) % 7 != 0:
        raise ValueError("Input list length must be divisible by 7")

    result = vals[:7]

    removed_frames = 0

    for i in range(7, len(vals), 7):
        current_first = vals[i]
        previous_first = result[-7]

        if current_first != previous_first:
            result.extend(vals[i:i+7])
        else:
            removed_frames += 1

    example["codes_list"] = result

    return example

ds = ds.map(remove_duplicate_frames, num_proc=num_proc)

tok_info = '''*** HERE you can modify the text prompt
i.e. if you wanted a multispeaker model like canopylabs/orpheus-3b-0.1-ft, you can pass:
f"{example["source"]}:  {example["text"]}", as is passed.
'''
print(tok_info)

def create_input_ids(example):
    if example.get("source") is not None:
        text_prompt = f"{example['source']}: {example['text']}"
    else:
        text_prompt = example["text"]
    text_ids = tokenizer.encode(text_prompt,  add_special_tokens=True)
    text_ids.append(end_of_text)
    example["text_tokens"] = text_ids
    input_ids = (
        [start_of_human]
        + example["text_tokens"]
        + [end_of_human]
        + [start_of_ai]
        + [start_of_speech]
        + example["codes_list"]
        + [end_of_speech]
        + [end_of_ai]
    )
    example["input_ids"] = input_ids
    example["labels"] = input_ids
    example["attention_mask"] = [1] * len(input_ids)

    return example

ds = ds.map(create_input_ids, num_proc=num_proc, remove_columns=["text", "codes_list"])

num_proc must be <= 29. Reducing num_proc to 29 for dataset of size 29.
Map (num_proc=29): 100%|██████████| 29/29 [00:02<00:00, 10.62 examples/s]
num_proc must be <= 29. Reducing num_proc to 29 for dataset of size 29.


*** HERE you can modify the text prompt
i.e. if you wanted a multispeaker model like canopylabs/orpheus-3b-0.1-ft, you can pass:
f"{example["source"]}:  {example["text"]}", as is passed.



Map (num_proc=29): 100%|██████████| 29/29 [00:02<00:00,  9.85 examples/s]


### Remove unnecessary columns

In [8]:
columns_to_keep = ["input_ids", "labels", "attention_mask"]
columns_to_remove = [col for col in ds.column_names if col not in columns_to_keep]

ds = ds.remove_columns(columns_to_remove)
ds.save_to_disk("data/raw_dataset")
ds

Saving the dataset (1/1 shards): 100%|██████████| 29/29 [00:00<00:00, 3312.95 examples/s]


Dataset({
    features: ['input_ids', 'labels', 'attention_mask'],
    num_rows: 29
})

## Train

In [9]:
from datasets import load_from_disk
ds = load_from_disk("data/raw_dataset")

max_seq_length=2244

# compute lengths for every example
lengths = [len(x) for x in ds['input_ids']]
max_len = max(lengths)
idx_max = lengths.index(max_len)
print("max length:", max_len, "at index", idx_max)
#  show a few long ones
long_indices = [i for i,l in enumerate(lengths) if l > max_seq_length]
print(f"examples > {max_seq_length}:", long_indices)

max length: 2244 at index 13
examples > 2244: []


In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "canopylabs/3b-es_it-ft-research_release",
    max_seq_length=max_seq_length, # Choose any for long context!
    dtype = None, # Select None for auto detection
    load_in_4bit = True, # Select True for 4bit which reduces memory usage
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
model.print_trainable_parameters()

==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA RTX 2000 Ada Generation Laptop GPU. Num GPUs = 1. Max memory: 7.653 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.74s/it]


canopylabs/3b-es_it-ft-research_release does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


Unsloth 2025.11.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


trainable params: 48,627,712 || all params: 3,349,494,784 || trainable%: 1.4518


In [ ]:
from transformers import TrainingArguments, Trainer
trainer = Trainer(
    model = model,
    train_dataset = ds,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 3,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = None,
        report_to = "none",
    ),
)

In [4]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA RTX 2000 Ada Generation Laptop GPU. Max memory = 7.653 GB.
3.131 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 29 | Num Epochs = 1 | Total steps = 3
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 48,627,712 of 3,349,494,784 (1.45% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,4.739200
2,4.728500
3,4.670000


In [6]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

NameError: name 'start_gpu_memory' is not defined

## Run Inference

In [ ]:
from unsloth import FastLanguageModel
import torch

checkpoint_path = "checkpoints/checkpoint-80"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = checkpoint_path,
    max_seq_length = 2250,
    dtype = None,
    load_in_4bit = False,
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/lpozzi/Git/data-science-lectures/tts_ita/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA RTX 2000 Ada Generation Laptop GPU. Num GPUs = 1. Max memory: 7.653 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.21s/it]


canopylabs/3b-es_it-ft-research_release does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


Unsloth 2025.11.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(156940, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [2]:
from snac import SNAC
snac_model = SNAC.from_pretrained("hubertsiuzdak/snac_24khz").to("cpu")

In [3]:
prompts = [
    # "Ora che hai fatto il pieno di informazioni utili, prenditi un momento per pensarci su e rispondi a questa domanda.",
    # "Un altro discorso celebre è quello che Proximo fa ai gladiatori, parlando della folla e del Colosseo.",
    "Signori si nasce e io, modestamente, lo nacqui."
]

chosen_voice = 'luca'

In [4]:
prompts_ = [(f"{chosen_voice}: " + p) if chosen_voice else p for p in prompts]

all_input_ids = []

for prompt in prompts_:
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids
  all_input_ids.append(input_ids)

start_token = torch.tensor([[ 128259]], dtype=torch.int64) # Start of human
end_tokens = torch.tensor([[128009, 128260]], dtype=torch.int64) # End of text, End of human

all_modified_input_ids = []
for input_ids in all_input_ids:
  modified_input_ids = torch.cat([start_token, input_ids, end_tokens], dim=1) # SOH SOT Text EOT EOH
  all_modified_input_ids.append(modified_input_ids)

all_padded_tensors = []
all_attention_masks = []
max_length = max([modified_input_ids.shape[1] for modified_input_ids in all_modified_input_ids])
for modified_input_ids in all_modified_input_ids:
  padding = max_length - modified_input_ids.shape[1]
  padded_tensor = torch.cat([torch.full((1, padding), 128263, dtype=torch.int64), modified_input_ids], dim=1)
  attention_mask = torch.cat([torch.zeros((1, padding), dtype=torch.int64), torch.ones((1, modified_input_ids.shape[1]), dtype=torch.int64)], dim=1)
  all_padded_tensors.append(padded_tensor)
  all_attention_masks.append(attention_mask)

all_padded_tensors = torch.cat(all_padded_tensors, dim=0)
all_attention_masks = torch.cat(all_attention_masks, dim=0)

input_ids = all_padded_tensors.to("cuda")
attention_mask = all_attention_masks.to("cuda")
generated_ids = model.generate(
      input_ids=input_ids,
      attention_mask=attention_mask,
      max_new_tokens=1200,
      do_sample=True,
      temperature=0.6,
      top_p=0.95,
      repetition_penalty=1.1,
      num_return_sequences=1,
      eos_token_id=128258,
     use_cache = True
  )
token_to_find = 128257
token_to_remove = 128258

token_indices = (generated_ids == token_to_find).nonzero(as_tuple=True)

if len(token_indices[1]) > 0:
    last_occurrence_idx = token_indices[1][-1].item()
    cropped_tensor = generated_ids[:, last_occurrence_idx+1:]
else:
    cropped_tensor = generated_ids

mask = cropped_tensor != token_to_remove

processed_rows = []

for row in cropped_tensor:
    masked_row = row[row != token_to_remove]
    processed_rows.append(masked_row)

code_lists = []

for row in processed_rows:
    row_length = row.size(0)
    new_length = (row_length // 7) * 7
    trimmed_row = row[:new_length]
    trimmed_row = [t - 128266 for t in trimmed_row]
    code_lists.append(trimmed_row)


def redistribute_codes(code_list):
  layer_1 = []
  layer_2 = []
  layer_3 = []
  for i in range((len(code_list)+1)//7):
    layer_1.append(code_list[7*i])
    layer_2.append(code_list[7*i+1]-4096)
    layer_3.append(code_list[7*i+2]-(2*4096))
    layer_3.append(code_list[7*i+3]-(3*4096))
    layer_2.append(code_list[7*i+4]-(4*4096))
    layer_3.append(code_list[7*i+5]-(5*4096))
    layer_3.append(code_list[7*i+6]-(6*4096))
  codes = [torch.tensor(layer_1).unsqueeze(0),
         torch.tensor(layer_2).unsqueeze(0),
         torch.tensor(layer_3).unsqueeze(0)]

  # codes = [c.to("cuda") for c in codes]
  audio_hat = snac_model.decode(codes)
  return audio_hat

my_samples = []
for code_list in code_lists:
  samples = redistribute_codes(code_list)
  my_samples.append(samples)
from IPython.display import display, Audio
if len(prompts) != len(my_samples):
  raise Exception("Number of prompts and samples do not match")
else:
  for i in range(len(my_samples)):
    print(prompts[i])
    samples = my_samples[i]
    display(Audio(samples.detach().squeeze().to("cpu").numpy(), rate=24000))
# Clean up to save RAM


Signori si nasce e io, modestamente, lo nacqui.


In [5]:
import soundfile as sf

samples_np = samples.detach().squeeze().cpu().numpy()
sf.write("data/luca/checkpoint_80.wav", samples_np, 24000)

In [5]:
# model.save_pretrained("lora_model")  # Local saving
# tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")
